Gender, ethnicity, Los, Age

In [1]:
import pandas as pd
import numpy as np

#from datetime import dt
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta


In [61]:
# Read 5 dataset
admission = pd.read_csv("mimic-iii-clinical-database-1.4/ADMISSIONS.csv") # 58976

admission = admission[["SUBJECT_ID","ADMITTIME", "DEATHTIME", "RELIGION", "ETHNICITY", "DISCHTIME"]]
patients = pd.read_csv("mimic-iii-clinical-database-1.4/PATIENTS.csv") #46520
patients = patients[["SUBJECT_ID","GENDER", "DOB", "DOD","EXPIRE_FLAG"]]
icustays = pd.read_csv("mimic-iii-clinical-database-1.4/ICUSTAYS.csv") #61532
icustays = icustays[["SUBJECT_ID", "LOS"]]
#services = pd.read_csv("mimic-iii-clinical-database-1.4/SERVICES.csv")
#transfer = pd.read_csv("mimic-iii-clinical-database-1.4/TRANSFERS.csv")


In [78]:
admission["ADMITTIME"] = pd.to_datetime(admission["ADMITTIME"]).dt.date # format = '%Y-%m-%d %H:%M:%S'
patients["DOB"] = pd.to_datetime(patients["DOB"]).dt.date

#admission["DISCHTIME"] = pd.to_datetime(admission["DISCHTIME"]).dt.date
#patients["DOD"] = pd.to_datetime(patients["DOD"]).dt.date

information = pd.merge(admission, patients, on = "SUBJECT_ID")

In [80]:
information.sort_values("SUBJECT_ID")

,SUBJECT_ID,ADMITTIME,DEATHTIME,RELIGION,ETHNICITY,DISCHTIME,GENDER,DOB,DOD,EXPIRE_FLAG
212,2,2138-07-17,NaN,NOT SPECIFIED,ASIAN,2138-07-21 15:48:00,M,2138-07-17,NaN,0
213,3,2101-10-20,NaN,CATHOLIC,WHITE,2101-10-31 13:58:00,M,2025-04-11,2102-06-14 00:00:00,1
214,4,2191-03-16,NaN,PROTESTANT QUAKER,WHITE,2191-03-23 18:41:00,F,2143-05-12,NaN,0
215,5,2103-02-02,NaN,BUDDHIST,ASIAN,2103-02-04 12:15:00,M,2103-02-02,NaN,0
216,6,2175-05-30,NaN,NOT SPECIFIED,WHITE,2175-06-15 16:00:00,F,2109-06-21,NaN,0
...,...,...,...,...,...,...,...,...,...,...
56436,99985,2181-01-27,NaN,JEWISH,WHITE,2181-02-12 17:05:00,M,2127-04-08,NaN,0
56437,99991,2184-12-24,NaN,NOT SPECIFIED,WHITE,2185-01-05 12:15:00,M,2137-04-07,NaN,0
56438,99992,2144-07-25,NaN,CATHOLIC,WHITE,2144-07-28 17:56:00,F,2078-10-17,NaN,0
56572,99995,2147-02-08,NaN,NOT SPECIFIED,WHITE,2147-02-11 13:15:00,F,2058-05-29,2147-09-29 00:00:00,1


In [72]:
information.loc[information['SUBJECT_ID'] == 23]

,SUBJECT_ID,ADMITTIME,DEATHTIME,RELIGION,ETHNICITY,DISCHTIME,GENDER,DOB,DOD,EXPIRE_FLAG
19,23,2157-10-18,None,CATHOLIC,WHITE,2157-10-25 14:00:00,M,2082-07-17,None,0


In [81]:
admission["ADMITTIME"] = pd.to_datetime(admission["ADMITTIME"]).dt.date # format = '%Y-%m-%d %H:%M:%S'
patients["DOB"] = pd.to_datetime(patients["DOB"]).dt.date

#admission["DISCHTIME"] = pd.to_datetime(admission["DISCHTIME"]).dt.date
#patients["DOD"] = pd.to_datetime(patients["DOD"]).dt.date

information = pd.merge(admission, patients, on = "SUBJECT_ID")

information = information.sort_values('ADMITTIME').groupby(["SUBJECT_ID"]).last().reset_index()

for index, row in information.iterrows():
    information.at[index,"AGE"] = relativedelta(row['ADMITTIME']  , row['DOB']).years
    #information.at[index,"LEAVE_TIME"] = relativedelta(row['DOD'], row['DISCHTIME']  ).years
    #print(row['ADMITTIME'], row['DOB'])
information = pd.merge(information, icustays,on="SUBJECT_ID")
#del admission, patients, icustays


information["DISCHTIME"] = pd.to_datetime(information['DISCHTIME']).dt.date
information["DOD"] = pd.to_datetime(information['DOD']).dt.date

# LEAVE TIME is the time lap between discharing and Date of Death
information["LEAVE_TIME"] = (information["DOD"] - information["DISCHTIME"] )

# OUTCOME is whether patients die within one year discharging 
# 1 means die and 0 means alive
information['OUTCOME'] =0
information.loc[information["LEAVE_TIME"]< pd.Timedelta(365, unit='d'), 'OUTCOME']=1
information.loc[information["LEAVE_TIME"]== pd.Timedelta(0, unit='d'), 'OUTCOME']=0

#information["LEAVE_TIME"] = information["LEAVE_TIME"].fillna(0)

information.loc[information["LEAVE_TIME"].isna(), 'LEAVE_TIME']=0

In [82]:
information.head(50)

,SUBJECT_ID,ADMITTIME,DEATHTIME,RELIGION,ETHNICITY,DISCHTIME,GENDER,DOB,DOD,EXPIRE_FLAG,AGE,LOS,LEAVE_TIME,OUTCOME
0,2,2138-07-17,None,NOT SPECIFIED,ASIAN,2138-07-21,M,2138-07-17,NaT,0,0.0,0.0918,0,0
1,3,2101-10-20,None,CATHOLIC,WHITE,2101-10-31,M,2025-04-11,2102-06-14,1,76.0,6.0646,226 days 00:00:00,1
2,4,2191-03-16,None,PROTESTANT QUAKER,WHITE,2191-03-23,F,2143-05-12,NaT,0,47.0,1.6785,0,0
3,5,2103-02-02,None,BUDDHIST,ASIAN,2103-02-04,M,2103-02-02,NaT,0,0.0,0.0844,0,0
4,6,2175-05-30,None,NOT SPECIFIED,WHITE,2175-06-15,F,2109-06-21,NaT,0,65.0,3.6729,0,0
5,7,2121-05-23,None,CATHOLIC,WHITE,2121-05-27,F,2121-05-23,NaT,0,0.0,0.2677,0,0
6,7,2121-05-23,None,CATHOLIC,WHITE,2121-05-27,F,2121-05-23,NaT,0,0.0,0.7391,0,0
7,8,2117-11-20,None,CATHOLIC,WHITE,2117-11-24,M,2117-11-20,NaT,0,0.0,1.0755,0,0
8,9,2149-11-09,2149-11-14 10:15:00,UNOBTAINABLE,UNKNOWN/NOT SPECIFIED,2149-11-14,M,2108-01-26,2149-11-14,1,41.0,5.3231,0 days 00:00:00,0
9,10,2103-06-28,None,UNOBTAINABLE,BLACK/AFRICAN AMERICAN,2103-07-06,F,2103-06-28,NaT,0,0.0,8.0921,0,0


In [21]:
information['OUTCOME'].unique()

array([0, 1])

In [83]:
information.loc[(information["ETHNICITY"]=="ASIAN")
                    |(information["ETHNICITY"]=="ASIAN - CHINESE")
                    |(information["ETHNICITY"]=="ASIAN - ASIAN INDIAN")
                    |(information["ETHNICITY"]== 'ASIAN - FILIPINO')
                    |(information["ETHNICITY"] =='ASIAN - VIETNAMESE')
                    |(information["ETHNICITY"] =='ASIAN - CAMBODIAN') 
                    |(information["ETHNICITY"] =='ASIAN - THAI')
                    |(information["ETHNICITY"] == 'ASIAN - OTHER')
                    |(information["ETHNICITY"] =="ASIAN - JAPANESE")
                    |(information["ETHNICITY"] =="ASIAN - KOREAN"), "ETHNICITY"] = "ASIAN"

information.loc[ (information["ETHNICITY"]=="BLACK/AFRICAN AMERICAN") 
                           |(information["ETHNICITY"]=='BLACK/AFRICAN')
                           |(information["ETHNICITY"]=="BLACK/CAPE VERDEAN")
                            |(information["ETHNICITY"]=='BLACK/HAITIAN'), "ETHNICITY"] = "BLACK"

information.loc[(information["ETHNICITY"]=='CARIBBEAN ISLAND'), 'ETHNICITY' ]= "CARIBBEAN_ISLAND"

information.loc[(information["ETHNICITY"]=='HISPANIC OR LATINO')
                      |(information["ETHNICITY"]=='HISPANIC/LATINO - GUATEMALAN')
                      |(information["ETHNICITY"]=='HISPANIC/LATINO - PUERTO RICAN')
                      |(information["ETHNICITY"]=='HISPANIC/LATINO - SALVADORAN')
                      |(information["ETHNICITY"]=='HISPANIC/LATINO - DOMINICAN')
                      |(information["ETHNICITY"]=='HISPANIC/LATINO - COLOMBIAN')
                      |(information["ETHNICITY"]=='HISPANIC/LATINO - HONDURAN')
                      |(information["ETHNICITY"]=='HISPANIC/LATINO - MEXICAN')
                      |(information["ETHNICITY"]=='HISPANIC/LATINO - CUBAN')
                      |(information["ETHNICITY"]== "HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)")
                      , "ETHNICITY"] = "HISPANIC"

information.loc[information["ETHNICITY"]=='SOUTH AMERICAN', 'ETHNICITY' ]= "SOUTH_AMERICAN"

information.loc[(information["ETHNICITY"]=='WHITE')
                |(information["ETHNICITY"]=='WHITE - RUSSIAN')
                |(information["ETHNICITY"]=='WHITE - OTHER EUROPEAN')
                |(information["ETHNICITY"]=='WHITE - EASTERN EUROPEAN')
                |(information["ETHNICITY"]=='WHITE - BRAZILIAN'),'ETHNICITY']= 'WHITE'

information.loc[(information["ETHNICITY"]=='PORTUGUESE'),'ETHNICITY'] = 'PORTUGUESE'
 
information.loc[(information["ETHNICITY"]=='MIDDLE EASTERN'),'ETHNICITY'] = 'MIDDLE_EASTERN'

information.loc[(information["ETHNICITY"]=='UNKNOWN/NOT SPECIFIED')
                |(information["ETHNICITY"]=='MULTI RACE ETHNICITY')
                |(information["ETHNICITY"]== 'UNABLE TO OBTAIN')
                |(information["ETHNICITY"]== 'PATIENT DECLINED TO ANSWER')
                | (information["ETHNICITY"]== 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER')
                | (information["ETHNICITY"]==  'AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE')
                | (information["ETHNICITY"]==  'AMERICAN INDIAN/ALASKA NATIVE'),'ETHNICITY'] = 'OTHER'

information.loc[(information["ETHNICITY"]=='CARIBBEAN_ISLAND') |(information["ETHNICITY"]=='BLACK'),"ETHNICITY" ] = "BLACK_CARIBBEAN"
information.loc[(information["ETHNICITY"]=='HISPANIC') |(information["ETHNICITY"]=='SOUTH_AMERICAN'),"ETHNICITY"] = 'HISPANIC_SAMERICAN'
information.loc[(information["ETHNICITY"]=='WHITE') |(information["ETHNICITY"]=='MIDDLE_EASTERN') |(information["ETHNICITY"]=='PORTUGUESE'),"ETHNICITY"] = "WHITE_ME_PORT"


In [87]:
information.head(20)

,SUBJECT_ID,ADMITTIME,DEATHTIME,RELIGION,ETHNICITY,DISCHTIME,GENDER,DOB,DOD,EXPIRE_FLAG,AGE,LOS,LEAVE_TIME,OUTCOME
0,2,2138-07-17,None,NOT SPECIFIED,ASIAN,2138-07-21,M,2138-07-17,NaT,0,0.0,0.0918,0,0
1,3,2101-10-20,None,CATHOLIC,WHITE_ME_PORT,2101-10-31,M,2025-04-11,2102-06-14,1,76.0,6.0646,226 days 00:00:00,1
2,4,2191-03-16,None,PROTESTANT QUAKER,WHITE_ME_PORT,2191-03-23,F,2143-05-12,NaT,0,47.0,1.6785,0,0
3,5,2103-02-02,None,BUDDHIST,ASIAN,2103-02-04,M,2103-02-02,NaT,0,0.0,0.0844,0,0
4,6,2175-05-30,None,NOT SPECIFIED,WHITE_ME_PORT,2175-06-15,F,2109-06-21,NaT,0,65.0,3.6729,0,0
5,7,2121-05-23,None,CATHOLIC,WHITE_ME_PORT,2121-05-27,F,2121-05-23,NaT,0,0.0,0.2677,0,0
6,7,2121-05-23,None,CATHOLIC,WHITE_ME_PORT,2121-05-27,F,2121-05-23,NaT,0,0.0,0.7391,0,0
7,8,2117-11-20,None,CATHOLIC,WHITE_ME_PORT,2117-11-24,M,2117-11-20,NaT,0,0.0,1.0755,0,0
9,10,2103-06-28,None,UNOBTAINABLE,BLACK_CARIBBEAN,2103-07-06,F,2103-06-28,NaT,0,0.0,8.0921,0,0
10,11,2178-04-16,None,OTHER,WHITE_ME_PORT,2178-05-11,F,2128-02-22,2178-11-14,1,50.0,1.5844,187 days 00:00:00,1


In [85]:
# Preprocess the databse 

# 1.Remove patients who died in the hospital (remove all outtime nonmissing value)
information = information[information["DEATHTIME"].isna()]
#information.dropna(subset=["DEATHTIME"])


# use other to represent missing values
information = information.fillna({'GENDER': 'other'})

# Patients age >89 have had their date of birth shifted to obscure their age
information.loc[information["AGE"] >89 , "AGE"] = 90


In [86]:
information.to_csv("information.csv")

In [27]:

# 2. Partitioned ethnicity into five categories



ASIAN = information[(information["ETHNICITY"]=="ASIAN")
                    |(information["ETHNICITY"]=="ASIAN - CHINESE")
                    |(information["ETHNICITY"]=="ASIAN - ASIAN INDIAN")
                    |(information["ETHNICITY"]== 'ASIAN - FILIPINO')
                    |(information["ETHNICITY"] =='ASIAN - VIETNAMESE')
                    |(information["ETHNICITY"] =='ASIAN - CAMBODIAN') 
                    |(information["ETHNICITY"] =='ASIAN - THAI')
                    |(information["ETHNICITY"] == 'ASIAN - OTHER')
                    |(information["ETHNICITY"] =="ASIAN - JAPANESE")
                    |(information["ETHNICITY"] =="ASIAN - KOREAN")]
BLACK_CB = information.loc[(information["ETHNICITY"]=='CARIBBEAN ISLAND') 
                           | (information["ETHNICITY"]=="BLACK/AFRICAN AMERICAN") 
                           |(information["ETHNICITY"]=='BLACK/AFRICAN')
                           |(information["ETHNICITY"]=="BLACK/CAPE VERDEAN")
                            |(information["ETHNICITY"]=='BLACK/HAITIAN')]
HISPANIC= information[(information["ETHNICITY"]=='HISPANIC OR LATINO')
                      |(information["ETHNICITY"]=='HISPANIC/LATINO - GUATEMALAN')
                      |(information["ETHNICITY"]=='HISPANIC/LATINO - PUERTO RICAN')
                      |(information["ETHNICITY"]=='HISPANIC/LATINO - SALVADORAN')
                      |(information["ETHNICITY"]=='HISPANIC/LATINO - DOMINICAN')
                      |(information["ETHNICITY"]=='HISPANIC/LATINO - COLOMBIAN')
                      |(information["ETHNICITY"]=='HISPANIC/LATINO - HONDURAN')
                      |(information["ETHNICITY"]=='HISPANIC/LATINO - MEXICAN')
                      |(information["ETHNICITY"]=='HISPANIC/LATINO - CUBAN')
                      |(information["ETHNICITY"]== "HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)")
                      |(information["ETHNICITY"]=='SOUTH AMERICAN')]
WHITE_ME_PO =information[(information["ETHNICITY"]=='WHITE')
                          |(information["ETHNICITY"]=='WHITE - RUSSIAN')
                          |(information["ETHNICITY"]=='WHITE - OTHER EUROPEAN')
                          |(information["ETHNICITY"]=='WHITE - EASTERN EUROPEAN')
                          |(information["ETHNICITY"]=='WHITE - BRAZILIAN')
                          |(information["ETHNICITY"]=='PORTUGUESE')
                          |(information["ETHNICITY"]=='MIDDLE EASTERN')
                          |(information["ETHNICITY"]=="AMERICAN INDIAN/ALASKA NATIVE")] 
OTHER = pd.concat([information,ASIAN, BLACK_CB,HISPANIC, WHITE_ME_PO]).drop_duplicates(keep=False)


In [28]:
information = pd.read_csv("information.csv")

In [29]:
import sklearn.preprocessing as preprocessing
from sklearn.preprocessing import StandardScaler
import numpy as np

In [32]:
y = information[["SUBJECT_ID",'OUTCOME']]

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)


/Users/Dhhh/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [ ]:
df1 = information[['ETHNICITY']]
labelEnc = preprocessing.LabelEncoder()
new_target = labelEnc.fit_transform(df1)
onehotEnc = preprocessing.OneHotEncoder()
onehotEnc.fit(new_target.reshape(-1, 1))
targets_trans2 = onehotEnc.transform(new_target.reshape(-1, 1))

#print(targets_trans.toarray())
target_trans = np.concatenate(targets_trans, targets_trans2.toarray(), axis = 1)

In [43]:
df2 = information[["SUBJECT_ID", 'GENDER','ETHNICITY','LOS','AGE', 'OUTCOME']]
X = pd.get_dummies(df2, columns=['GENDER','ETHNICITY'])
#df2[df2["OUTCOME"]==1]

X[["LOS", "AGE"]]=StandardScaler().fit_transform(X[["LOS", "AGE"]])
X

,SUBJECT_ID,LOS,AGE,OUTCOME,GENDER_F,GENDER_M,ETHNICITY_ASIAN,ETHNICITY_BLACK_CARIBBEAN,ETHNICITY_HISPANIC_SAMERICAN,ETHNICITY_OTHER,ETHNICITY_WHITE_ME_PORT
0,22,-0.396618,0.348668,0,1,0,0,0,0,0,1
1,23,-0.382266,0.646807,0,0,1,0,0,0,0,1
2,23,-0.391559,0.646807,0,0,1,0,0,0,0,1
3,23,-0.382266,0.817172,0,0,1,0,0,0,0,1
4,23,-0.391559,0.817172,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
107073,98790,-0.482122,-0.119837,0,0,1,0,0,0,0,1
107074,98790,-0.193922,-0.119837,0,0,1,0,0,0,0,1
107075,98794,-0.260016,0.944946,0,0,1,0,0,0,0,1
107076,98800,-0.428352,-1.567941,0,1,0,0,0,0,0,1


In [47]:
X.to_csv("X.csv", index = False)
y.to_csv("y.csv", index = False)